In [1]:
%%javascript
IPython.load_extensions('calico-document-tools');

<IPython.core.display.Javascript object>

#Imports and function definitions from 150311_calc_power_in_a_layer.ipynb 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline
from IPython.display import Image
from __future__ import division
from __future__ import print_function

degtorad = np.pi/180.

# Angle units are assumed to be in radians

def cos_angle2(angle1, n1, n2): # From Snell's law
    return np.sqrt(1 - (np.sin(angle1)*n1/n2)**2)

def reflectance_TE(angle1, n1, n2):
    c1 = np.cos(angle1)
    c2 = cos_angle2(angle1, n1, n2)
    temp = (n1*c1 - n2*c2) / (n1*c1 + n2*c2)
    return temp**2

def transmittance_TE(angle1, n1, n2):
    return 1.0 - reflectance_TE(angle1, n1, n2)

def reflectance_TM(angle1, n1, n2):
    c1 = np.cos(angle1)
    c2 = cos_angle2(angle1, n1, n2)
    temp = (n1*c2 - n2*c1) / (n1*c2 + n2*c1)
    return temp**2

def transmittance_TM(angle1, n1, n2):
    return 1.0 - reflectance_TM(angle1, n1, n2)

def reflectance_Unpol(angle1, n1, n2):
    rTE = reflectance_TE(angle1, n1, n2)
    rTM = reflectance_TM(angle1, n1, n2)
    return 0.5 * (rTE + rTM)

def transmittance_Unpol(angle1, n1, n2):
    return 1.0 - reflectance_Unpol(angle1, n1, n2)

# Given a list of refractive indices of layers, calculate the propagation angle
# in any given layer. Incidence medium is layer 0, exit medium is layer
# len(refr_indices)-1. All angles are in radians.

def maxindex(a):
    return len(a) - 1

def incidence_angle_in_layer_i(i, angle0, refr_indices):
    if i > maxindex(refr_indices):
        print('INDEX OUT OF RANGE in function incidence_angle_in_layer_i')
        return
    elif i == 0:
        return angle0
    else:
        return np.arcsin( np.sin(angle0) * refr_indices[0] / refr_indices[i] )

# Calculate power for ray angle specified direction in layer 0 once
# ray reaches the layer given by "layerindex"

def power_in_specified_direction(angle0, refr_indices, layerindex):
    if layerindex > maxindex(refr_indices):
        print('LAYERINDEX OUT OF RANGE in function power_in_specified_direction')
        return
    temp = 1.0
    for i in range(1,layerindex+1):
        angle = incidence_angle_in_layer_i(i, angle0, refr_indices)
        temp *= transmittance_Unpol(angle, refr_indices[i-1], refr_indices[i])
    return temp

In [3]:
refr_indices = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]

angle0 = 88.0
for i in range(1,len(refr_indices)):
    angle = incidence_angle_in_layer_i(i, angle0*degtorad, refr_indices)
    print(i, refr_indices[i], angle/degtorad, 
          transmittance_Unpol(angle, refr_indices[i-1], refr_indices[i]), 
          power_in_specified_direction(angle0*degtorad, refr_indices, i))

1 1.5 41.7791042687 0.952918658155 0.952918658155
2 2.0 29.9798507651 0.97853066309 0.932460126436
3 2.5 23.5629464258 0.987393631132 0.920705190127
4 3.0 19.4588810158 0.991650843665 0.913018078556
5 3.5 16.5911438044 0.994049771163 0.907585412056
6 4.0 14.4685004425 0.995540836183 0.903538340026


#Problem with calculation formulation 

There is a problem with calculating the power in the direction of a particular ray for unpolarized light. The correct approach is to calculate the TE and TM power individually to that layer, and then average them instead of calculating the average in each layer and using that to get the power in the subsequent layers.

###RESTART KERNEL

# Fix problem - Use these functions!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline
from IPython.display import Image
from __future__ import division
from __future__ import print_function

degtorad = np.pi/180.

# Angle units are assumed to be in radians

def cos_angle2(angle1, n1, n2): # From Snell's law
    return np.sqrt(1 - (np.sin(angle1)*n1/n2)**2)

def reflectance_TE(angle1, n1, n2):
    c1 = np.cos(angle1)
    c2 = cos_angle2(angle1, n1, n2)
    temp = (n1*c1 - n2*c2) / (n1*c1 + n2*c2)
    return temp**2

def transmittance_TE(angle1, n1, n2):
    return 1.0 - reflectance_TE(angle1, n1, n2)

def reflectance_TM(angle1, n1, n2):
    c1 = np.cos(angle1)
    c2 = cos_angle2(angle1, n1, n2)
    temp = (n1*c2 - n2*c1) / (n1*c2 + n2*c1)
    return temp**2

def transmittance_TM(angle1, n1, n2):
    return 1.0 - reflectance_TM(angle1, n1, n2)

def reflectance_Unpol(angle1, n1, n2):
    rTE = reflectance_TE(angle1, n1, n2)
    rTM = reflectance_TM(angle1, n1, n2)
    return 0.5 * (rTE + rTM)

def transmittance_Unpol(angle1, n1, n2):
    return 1.0 - reflectance_Unpol(angle1, n1, n2)

# Given a list of refractive indices of layers, calculate the propagation angle
# in any given layer. Incidence medium is layer 0, exit medium is layer
# len(refr_indices)-1. All angles are in radians.

def maxindex(a):
    return len(a) - 1

def incidence_angle_in_layer_i(i, angle0, refr_indices):
    if i > maxindex(refr_indices):
        print('INDEX OUT OF RANGE in function incidence_angle_in_layer_i')
        return
    elif i == 0:
        return angle0
    else:
        return np.arcsin( np.sin(angle0) * refr_indices[0] / refr_indices[i] )

# Calculate power for ray angle specified direction in layer 0 once
# ray reaches the layer given by "layerindex"

def power_in_specified_direction(angle0, refr_indices, layerindex, pol='Unpol'):
    if layerindex > maxindex(refr_indices):
        print('LAYERINDEX OUT OF RANGE in function power_in_specified_direction')
        return
    powerTE = 1.0
    powerTM = 1.0
    for i in range(1,layerindex+1):
        angle = incidence_angle_in_layer_i(i, angle0, refr_indices)
        powerTE *= transmittance_TE(angle, refr_indices[i-1], refr_indices[i])
        powerTM *= transmittance_TM(angle, refr_indices[i-1], refr_indices[i])
    if pol == 'Unpol':
        temp = 0.5 * (powerTE + powerTM)
    elif pol == 'TE':
        temp = powerTE
    elif pol == 'TM':
        temp = powerTM
    else:
        print('INVALID POLARIZATION SPECIFIED in function incidence_angle_in_layer_i')
    return temp

##Test fixed  power_in_specified_direction function

In [5]:
refr_indices = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]

angle0 = 88.0
print('Unpolarized')
for i in range(1,len(refr_indices)):
    angle = incidence_angle_in_layer_i(i, angle0*degtorad, refr_indices)
    print(i, refr_indices[i], angle/degtorad, 
          transmittance_Unpol(angle, refr_indices[i-1], refr_indices[i]), 
          power_in_specified_direction(angle0*degtorad, refr_indices, i))
    
print('\nTE polarized')
for i in range(1,len(refr_indices)):
    angle = incidence_angle_in_layer_i(i, angle0*degtorad, refr_indices)
    print(i, refr_indices[i], angle/degtorad, 
          transmittance_TE(angle, refr_indices[i-1], refr_indices[i]), 
          power_in_specified_direction(angle0*degtorad, refr_indices, i, pol='TE'))
    
print('\nTM polarized')
for i in range(1,len(refr_indices)):
    angle = incidence_angle_in_layer_i(i, angle0*degtorad, refr_indices)
    print(i, refr_indices[i], angle/degtorad, 
          transmittance_TM(angle, refr_indices[i-1], refr_indices[i]), 
          power_in_specified_direction(angle0*degtorad, refr_indices, i, pol='TM'))


Unpolarized
1 1.5 41.7791042687 0.952918658155 0.952918658155
2 2.0 29.9798507651 0.97853066309 0.932791249894
3 2.5 23.5629464258 0.987393631132 0.921188956815
4 3.0 19.4588810158 0.991650843665 0.913575440212
5 3.5 16.5911438044 0.994049771163 0.90818144936
6 4.0 14.4685004425 0.995540836183 0.904156237246

TE polarized
1 1.5 41.7791042687 0.918051239769 0.918051239769
2 2.0 29.9798507651 0.969034019119 0.88962288263
3 2.5 23.5629464258 0.983760936549 0.875176240192
4 3.0 19.4588810158 0.989963614195 0.866392633798
5 3.5 16.5911438044 0.993159784313 0.860466321313
6 4.0 14.4685004425 0.995027000314 0.856187222567

TM polarized
1 1.5 41.7791042687 0.987786076542 0.987786076542
2 2.0 29.9798507651 0.988027307061 0.975959617158
3 2.5 23.5629464258 0.991026325714 0.967201673438
4 3.0 19.4588810158 0.993338073136 0.960758246626
5 3.5 16.5911438044 0.994939758012 0.955896577407
6 4.0 14.4685004425 0.996054672052 0.952125251924


In [6]:
print( 0.5 * (0.856187222567 + 0.952125251924) )

0.904156237245
